In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from RiskLib import calc_returns, value_at_risk, ew_cov, simulation

In [2]:
data = pd.read_csv("VaR.csv")
data.head()

,Date,SPY,AAPL,MSFT,AMZN,NVDA,GOOGL,TSLA,GOOG,BRK-B,...,CI,ETN,SLB,PGR,SCHW,LRCX,ZTS,C,BSX,AMT
0,2022-09-01,388.679565,157.028442,257.908386,127.820000,139.243423,109.739998,277.160004,110.550003,282.429993,...,282.124664,136.138718,36.036259,123.325203,70.653595,424.766022,158.011505,46.662727,40.990002,249.972107
1,2022-09-02,384.581177,154.891113,253.609909,127.510002,136.346054,107.849998,270.209991,108.680000,277.670013,...,281.791626,134.696564,37.229321,122.039116,69.294113,422.966797,155.602798,46.748875,40.490002,243.384186
2,2022-09-06,383.130066,153.618668,250.826813,126.110001,134.527710,106.809998,274.420013,107.480003,276.589996,...,280.483856,135.834595,36.703716,122.108894,69.422180,419.574921,155.464035,46.155418,40.380001,246.615067
3,2022-09-07,390.013000,155.040237,255.620483,129.479996,137.056152,109.449997,283.700012,110.480003,281.040009,...,285.154480,138.591293,36.408989,126.814598,70.722557,424.726715,158.992813,46.547867,41.470001,251.776749
4,2022-09-08,392.562256,153.549088,256.046387,129.820007,139.814484,108.379997,289.260010,109.419998,283.339996,...,285.852631,140.523956,37.499485,128.828476,72.072182,431.766174,157.099579,47.763493,42.709999,253.882172


In [3]:
arithmetic_returns = calc_returns.return_calculate(data, "DISCRETE", "Date")
arithmetic_returns = arithmetic_returns.drop(arithmetic_returns.columns[0], axis=1)

In [4]:
arithmetic_returns['META'].head()

0   -0.030479
1   -0.011103
2    0.011669
3    0.010412
4    0.043749
Name: META, dtype: float64

In [5]:
# Using a normal distribution
var_normal = value_at_risk.var_normal(arithmetic_returns['META'])
print(var_normal)

(0.05149979607637217, 0.05428693242254699)


In [6]:
# Using a normal distribution with an Exponentially Weighted variance
var_normal_EW = value_at_risk.var_normal_ew(arithmetic_returns['META'])
print(var_normal_EW)

(0.05149979607637217, 0.0016978649735826768)


In [7]:
# Using a MLE fitted T distribution
var_mle_t = value_at_risk.var_t_dist(arithmetic_returns['META'])
print(var_mle_t)

(0.040347050386296476, 0.04313418673247129)


In [8]:
# Using a fitted AR(1) model
var_ar1 = value_at_risk.var_ar(arithmetic_returns['META'])
print(var_ar1)

(array([0.05091861]), array([0.05370575]))


In [9]:
# Using a Historic Simulation
var_historic = value_at_risk.var_historic(arithmetic_returns['META'])
print(var_historic)

(0.03669711360916308, 0.03948424995533789)


In [10]:
portfolio = pd.read_csv("portfolio.csv")
portfolio.head()

,Portfolio,Stock,Holding
0,A,AAPL,158
1,A,MSFT,178
2,A,AMZN,110
3,A,NVDA,54
4,A,GOOGL,69


In [11]:
stocks_in_portfolio_A = portfolio['Stock'][portfolio['Portfolio'] == 'A'].values
stocks_in_portfolio_B = portfolio['Stock'][portfolio['Portfolio'] == 'B'].values
stocks_in_portfolio_C = portfolio['Stock'][portfolio['Portfolio'] == 'C'].values

portfolio_A_values = portfolio['Holding'][portfolio['Portfolio'] == 'A'].values * data[stocks_in_portfolio_A].iloc[-1]
portfolio_B_values = portfolio['Holding'][portfolio['Portfolio'] == 'B'].values * data[stocks_in_portfolio_B].iloc[-1]
portfolio_C_values = portfolio['Holding'][portfolio['Portfolio'] == 'C'].values * data[stocks_in_portfolio_C].iloc[-1]

portfolio_total_values = portfolio['Holding'].values * data[portfolio['Stock']].iloc[-1]

portfolio_A_weights = portfolio_A_values / portfolio_A_values.sum()
portfolio_B_weights = portfolio_B_values / portfolio_B_values.sum()
portfolio_C_weights = portfolio_C_values / portfolio_C_values.sum()
portfolio_total_weights = portfolio_total_values / portfolio_total_values.sum()

In [15]:
portfolio_A_returns = arithmetic_returns[stocks_in_portfolio_A]
portfolio_B_returns = arithmetic_returns[stocks_in_portfolio_B]
portfolio_C_returns = arithmetic_returns[stocks_in_portfolio_C]
portfolio_total_returns = arithmetic_returns[portfolio['Stock']]

portfolio_A_ew_cov = ew_cov.ewCovar(portfolio_A_returns, 0.94)
portfolio_B_ew_cov = ew_cov.ewCovar(portfolio_B_returns, 0.94)
portfolio_C_ew_cov = ew_cov.ewCovar(portfolio_C_returns, 0.94)
portfolio_total_ew_cov = ew_cov.ewCovar(portfolio_total_returns, 0.94)

simulated_A_returns = simulation.pca_simulation(portfolio_A_ew_cov)
simulated_B_returns = simulation.pca_simulation(portfolio_B_ew_cov)
simulated_C_returns = simulation.pca_simulation(portfolio_C_ew_cov)
simulated_total_returns = simulation.pca_simulation(portfolio_total_ew_cov)

simulated_portfolioA_return = np.dot(simulated_A_returns, portfolio_A_weights)
simulated_portfolioB_return = np.dot(simulated_B_returns, portfolio_B_weights)
simulated_portfolioC_return = np.dot(simulated_C_returns, portfolio_C_weights)
simulated_portfolio_total_return = np.dot(simulated_total_returns, portfolio_total_weights)

simulated_portfolioA_value_changes = simulated_portfolioA_return * portfolio_A_values.sum()
simulated_portfolioB_value_changes = simulated_portfolioB_return * portfolio_B_values.sum()
simulated_portfolioC_value_changes = simulated_portfolioC_return * portfolio_C_values.sum()
simulated_portfolio_total_value_changes = simulated_portfolio_total_return * portfolio_total_values.sum()

confidence_level = 0.95

portfolioA_var = -np.percentile(simulated_portfolioA_value_changes, (1 - confidence_level) * 100)
print(f"The {confidence_level * 100}% VaR of the portfolio A is ${portfolioA_var:.2f}")
portfolioB_var = -np.percentile(simulated_portfolioB_value_changes, (1 - confidence_level) * 100)
print(f"The {confidence_level * 100}% VaR of the portfolio B is ${portfolioB_var:.2f}")
portfolioC_var = -np.percentile(simulated_portfolioC_value_changes, (1 - confidence_level) * 100)
print(f"The {confidence_level * 100}% VaR of the portfolio C is ${portfolioC_var:.2f}")
portfolio_total_var = -np.percentile(simulated_portfolio_total_value_changes, (1 - confidence_level) * 100)
print(f"The {confidence_level * 100}% VaR of the portfolio C is ${portfolio_total_var:.2f}")

[[ 0.00012245  0.00090231  0.00107416 ...  0.01508764 -0.00137812
  -0.00276684]
 [ 0.00094241  0.00257823 -0.00403511 ... -0.01516286  0.00025161
   0.00980351]
 [ 0.00608157 -0.01476465 -0.02123456 ...  0.00632574 -0.01137736
  -0.01302832]
 ...
 [ 0.00642874 -0.00761164  0.008168   ...  0.00505039  0.00810648
   0.01146737]
 [-0.00023506 -0.00578266 -0.0137033  ...  0.0237076  -0.00508348
   0.01247314]
 [ 0.05049009  0.00871253  0.00089898 ...  0.00072369  0.02512261
   0.01980558]]
The 95.0% VaR of the portfolio A is $15476.87
The 95.0% VaR of the portfolio B is $8110.79
The 95.0% VaR of the portfolio C is $18193.69
The 95.0% VaR of the portfolio C is $38988.24
